In [1]:
import random
import numpy as np

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

class SchellingAgent(Agent):
    '''
    Schelling segregation agent
    '''
    def __init__(self, pos, model, agent_type):
        '''
        Create a new Schelling agent.

        Args:
            unique_id: Unique identifier for the agent.
            x, y: Agent initial location.
            agent_type: Indicator for the agent's type (minority=1, majority=0)
        '''
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type
        
        #According to the agent type, we assign a wealth parameter (income). We assume that the first type agents
        #are rich and the second one are poor. Based on that, we update the total wealth of agents.
        if self.type == 0:
            self.wealth = np.random.uniform(0.5, 1)
        else:
            self.wealth = np.random.uniform(0.0, 0.5)
        self.model.total_agent_wealth = self.model.total_agent_wealth + self.wealth

    def step(self):
        #At first, we check if the agent is rich enough to stay at the assigned house - if not, he's moved out of the
        #grid. Then, we check if he's surrounded by similar neighbors - if not, he's out once again. Finally, we
        #consider the probability of him/her moving out nonetheless - an adjustment to Eq (5) ->
        #probability_of_moving_out = 1.5 - neighborhood_status + income_of_agent
        ###NOTE: I changed income_gap to income_of_agent for simplicity reasons###
        
        #Check if property is affortable
        if self.wealth >= self.model.property_prices(self.pos):
            
            #Find the number of similar similar neighbors
            similar = 0
            for neighbor in self.model.grid.neighbor_iter(self.pos):
                if neighbor.type == self.type:
                    similar += 1
            
            #Check if his/her neighbors are similar enough
            if similar > self.model.homophily:
                
                #For the calculation of the probability, assume a uniform distribution and pick a random number.
                #If that number is higher than an arbitrary value within the distribution, then the agent leaves.
                #We've decided to set that arbitrary value at 1.4 in order to have the less agents possible who
                #leave because of that random factor.
                moving_out_prob = np.random.uniform(0, 1.5 - self.model.neighborhood_status + self.wealth)
                if moving_out_prob < 1.4:
                    self.model.happy += 1
                else:
                    #Bye, Felicia
                    self.model.remove_agent(self)
                
            else:
                #Neighbors are not similar enough
                self.model.remove_agent(self)

        else:
            #Property's not affortable
            self.model.remove_agent(self)
        

class SchellingModel(Model):
    '''
    Model class for the Schelling segregation model.
    '''

    def __init__(self, height, width, density, minority_pc, homophily):
        '''
        '''

        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc
        self.homophily = homophily

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(height, width, torus=True)
        
        #Initialize the total agent wealth
        self.total_agent_wealth = 0
        
        #set condition of properties
        self.property_condition = np.random.uniform((width,height))
        
        #set deprication rate
        self.deprication_rate = 0.0028
        
        #Initialize neighborhood status based on the gaussian distribution found on Eq (19) 
        self.neighborhood_status = np.random.normal(0, 0.025) 
        
        # set prices of properties
        self.property_prices = np.random.uniform((width,height))
        
        self.happy = 0

        self.datacollector = DataCollector(
            model_reporters={"happy": lambda self: self.happy},
            agent_reporters={"x": lambda a: a.pos[0], "y": lambda a: a.pos[1]})

        self.running = True

        # Set up agents
        # We use a grid iterator that returns
        # the coordinates of a cell as well as
        # its contents. (coord_iter)
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            if random.random() < self.density:
                if random.random() < self.minority_pc:
                    agent_type = 1
                else:
                    agent_type = 0

                agent = SchellingAgent((x, y), self, agent_type)
                self.grid.position_agent(agent, (x, y))
                self.schedule.add(agent)

    def step(self):
        '''
        Run one step of the model. If All agents are happy, halt the model.
        '''
        self.happy = 0  # Reset counter of happy agents
        self.schedule.step()
        self.datacollector.collect(self)
        
        #In each step, the condition of each property is updated based on the owner's wealth 
        #and the depreciation rate. If the owner's rich, there's a slight improvement on the condition of
        #the property. Otherwise, it becomes a dump even faster.
        if self.agent.wealth > 1/2:
            self.property_condition = self.property_condition + self.agent.wealth / 10 - self.depriciation_rate
        else:
            self.property_condition = self.property_condition - self.agent.wealth / 10 - self.depriciation_rate
            
        #In each step, the price of each property is updated, too. The price is updated according to Eq (6) ->
        #property_price = 1/2 * (property_condition + mean(total_agent_wealth))
        self.property_price = 1/2 * (self.property_condition + np.mean(self.total_agent_wealth))
        
        #In each step, we update the neighborhood status based on Eq (19), which takes into consideration
        #the condition of each property, its owner's income and a gaussian distribution.
        self.neighborhood_status = self.neighborhood_status + 1 / self.schedule.get_agent_count() * (sum(self.property_condition) + self.total_agent_wealth) + np.random.normal(0, 0.025) 
        
        if self.happy == self.schedule.get_agent_count():
            self.running = False


In [2]:
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.UserParam import UserSettableParameter

from mesa.visualization.TextVisualization import (
    TextData, TextGrid, TextVisualization
)

from model import SchellingModel


class SchellingTextVisualization(TextVisualization):
    '''
    ASCII visualization for schelling model
    '''

    def __init__(self, model):
        '''
        Create new Schelling ASCII visualization.
        '''
        self.model = model

        grid_viz = TextGrid(self.model.grid, self.ascii_agent)
        happy_viz = TextData(self.model, 'happy')
        self.elements = [grid_viz, happy_viz]

    @staticmethod
    def ascii_agent(a):
        '''
        Minority agents are X, Majority are O.
        '''
        if a.type == 0:
            return 'O'
        if a.type == 1:
            return 'X'


class HappyElement(TextElement):
    '''
    Display a text count of how many happy agents there are.
    '''

    def __init__(self):
        pass

    def render(self, model):
        return "Happy agents: " + str(model.happy)


def schelling_draw(agent):
    '''
    Portrayal Method for canvas
    '''

    portrayal = {"Shape": "rect", "h": 1.0, "w": 1.0 ,"Filled": "true", "Layer": 0}

    if agent.type == 0:
        portrayal["Color"] = "Red"
    else:
        portrayal["Color"] = "Green"
    if agent is None:
        portrayal["Color"] = "Black"
        #return
    return portrayal


happy_element = HappyElement()
canvas_element = CanvasGrid(schelling_draw, 50, 50, 500, 500)
happy_chart = ChartModule([{"Label": "happy", "Color": "Black"}])

model_params = {
    "height": 50,
    "width": 50,
    "density": UserSettableParameter("slider", "Agent density", 0.9, 0.1, 1.0, 0.05),
    "minority_pc": UserSettableParameter("slider", "Fraction minority", 0.5, 0.00, 1.0, 0.05),
    "homophily": UserSettableParameter("slider", "Homophily", 5, 0, 8, 1)
}

server = ModularServer(SchellingModel,
                       [canvas_element, happy_element, happy_chart],
                       "Schelling", model_params)
server.launch()


Interface starting at http://127.0.0.1:8521
Socket opened!
{"type":"get_params"}
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
